In [ ]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import Select
import pandas as pd
import time
website = "https://www.adamchoi.co.uk/overs/detailed"

# Full path to the ChromeDriver executable
path = "C:\\Users\\UmirzakhToshmurodov\\Downloads\\chromedriver-win64\\chromedriver.exe"

# Use Service object
service = Service(path)
driver = webdriver.Chrome(service=service)

driver.get(website)

all_matches_button = driver.find_element(By.XPATH, '//label[@analytics-event="All matches"]')
all_matches_button.click()

# dropdown part
dropdown = Select(driver.find_element(By.ID, 'country'))
dropdown.select_by_visible_text('Spain')


time.sleep(3)


football_matches = driver.find_elements(By.TAG_NAME, "tr")

date = []
home_team = []
score = []
away_team = []

for match in football_matches:
    date.append(match.find_element(By.XPATH, './td[1]').text)
    home = match.find_element(By.XPATH, './td[3]').text
    home_team.append(home)
    print(home)
    score.append(match.find_element(By.XPATH, './td[4]').text)
    away_team.append(match.find_element(By.XPATH, './td[5]').text)
driver.quit()

df = pd.DataFrame({'date': date, 'home_team':home_team, 'score':score, 'away_team':away_team})    
df.to_csv('La_liga_2024-2025.csv', index=False)

Alaves
Valladolid
Alaves
Ath Bilbao
Alaves
Alaves
Sevilla
Alaves
Girona
Alaves
Las Palmas
Alaves
Mallorca
Alaves
Leganes
Alaves
FC Barcelona
Alaves
Betis
Alaves
Valencia
Alaves
Osasuna
Alaves
Ath Madrid
Villarreal
Alaves
Vallecano
Alaves
Alaves
Getafe
Real Madrid
Alaves
Espanol
Alaves
Sociedad
Alaves
Celta
Ath Bilbao
Valencia
Getafe
Ath Bilbao
Sociedad
Ath Bilbao
Real Madrid
Ath Bilbao
Villarreal
Ath Bilbao
Sevilla
Ath Bilbao
Ath Madrid
Ath Bilbao
Espanol
Ath Bilbao
Betis
Ath Bilbao
Celta
Osasuna
Alaves
Ath Bilbao
Ath Bilbao
Vallecano
Ath Bilbao
Valladolid
Ath Bilbao
Mallorca
Ath Bilbao
Girona
Ath Bilbao
Ath Bilbao
Leganes
Las Palmas
Ath Bilbao
Ath Bilbao
FC Barcelona
Ath Bilbao
Girona
Ath Madrid
Osasuna
Ath Madrid
Alaves
Ath Madrid
Las Palmas
Ath Madrid
Sevilla
Espanol
Ath Madrid
Getafe
Ath Madrid
Valencia
Ath Madrid
Real Madrid
Ath Madrid
Ath Madrid
Leganes
Ath Madrid
FC Barcelona
Ath Madrid
Ath Madrid
Valladolid
Ath Madrid
Mallorca
Ath Madrid
Betis
Ath Madrid
Sociedad
Ath Madrid
Cel